# Numerical solution of the Kuramoto-Sivashinsky Equation using Fourier spectral methods

Equations are taken from this paper: https://engineeringresearch.org/index.php/GJRE/article/view/1140/1072

In [1]:
using DifferentialEquations,FFTW,Plots, LSODA

In [2]:
N = 256
L = π
ν = 0.021 #0.02
k = ifftshift(range(-N÷2,N÷2-1,N))
k2 = @. k^2
k4 = @. k^4
x = range(-L,L,N)
tau = 1.05;

In [3]:
function Kuramoto!(du, u, p, t)
    N, L = p
    u_   = fft(u)
    u_2  = fft(u.^2)
    duk = @. (k2 - ν*k4)*u_-im*k/2*u_2
    du .= real.(ifft(duk))
end

Kuramoto! (generic function with 1 method)

In [4]:
u0 = -sin.(x)
p = [N,L]
tspan = (0.0, 20*tau);

plot(x,u0,legend=false)
xlabel!("x")
ylabel!("u")
title!("Kezdeti értékek")
savefig("Initial.pdf")

In [5]:
prob = ODEProblem(Kuramoto!, u0, tspan, p);

In [6]:
sol = solve(prob,lsoda(),dt=1e-4,saveat=1e-4);#tspan[2]/5e4);

In [7]:
E(u) = sum(u .* u)^0.5;

In [8]:
u = hcat(real.(sol.u)...);

In [9]:
Energy = E.(sol.u);

In [10]:
dE = (Energy .- circshift(Energy,-1))./1e-4;

In [11]:
plot(sol.t, Energy,label="ν=0.021")
xlabel!("t")
ylabel!("E")
title!("Kaotikus megoldás energiája")
savefig("Energy_chaotic_2.pdf")

"/home/nadorij/Elementary_chaos_theory/Energy_chaotic_2.pdf"

In [12]:
plot(Energy[1:end-1],dE[1:end-1],label="ν=0.021")
xlabel!("E")
ylabel!("dE/dt")
title!("Kaotikus megoldás fázistere")
savefig("Phase_plane_chaotic_2.pdf")

"/home/nadorij/Elementary_chaos_theory/Phase_plane_chaotic_2.pdf"

plot(Energy[1:end-1],dE[1:end-1],sol.t[1:end-1],label="ν=0.05524")
xlabel!("E")
ylabel!("dE/dt")
zlabel!("t")
title!("Periódikus megoldás fázistere")
savefig("Phase_plane_period_3D.pdf")

In [13]:
a = []
offset = (0)*1e-4
for idx in 1:length(Energy)
    if (((sol.t[idx]-offset)%tau)<1e-4) & (sol.t[idx]>offset-1e-4)
        append!(a,idx)
    end
end

In [14]:
scatter(Energy[a],dE[a])
xlabel!("E")
ylabel!("dE/dt")
title!("Periódikus megoldás stroboszkopikus leképzése")
savefig("Phase_plane_chaotic_stroboszkipic_2.pdf")

"/home/nadorij/Elementary_chaos_theory/Phase_plane_chaotic_stroboszkipic_2.pdf"

In [13]:
plot(sol.t,Energy)
#scatter!([sol.t[26284+6000]],[Energy[26284+6000]])

In [15]:
heatmap(sol.t[a],x,u[1:end,a],c=:viridis,xlabel="t",ylabel="x")
savefig("chaotic_heatmap_2.pdf") 

"/home/nadorij/Elementary_chaos_theory/chaotic_heatmap_2.pdf"

In [15]:
plot(x,ones(N).*sol.t[1],sol.u[1],zlim=(-1,4), legend = false, colorbar=true,line_z = sol.u[1],color=:gist_rainbow,dpi=300,size=(1000,1000))
for i in 1:length(sol.t)
    plot!(x,ones(N).*sol.t[i],sol.u[i], legend = false, colorbar=true, line_z = sol.u[i],color=:gist_rainbow,dpi=300,size=(1000,1000))
end
savefig("myplot_rainbow.png") 

"/home/nadorij/Elementary_chaos_theory/myplot_rainbow.png"

In [19]:
tspan = (0.0, 1e-4)

(0.0, 0.0001)

In [89]:
λ = []
function Lyapunov!(λ)
    T = 10000
    u_ = u0 
    for time in 1:T
        tspan = (0.0,1e-3)
        prob1 = ODEProblem(Kuramoto!, u_, tspan, p)
        sol1 = solve(prob1,lsoda(),dt=1e-3,saveat=1e-3)
        u_l = u_ .+ sqrt(0.001/N)
        prob2 = ODEProblem(Kuramoto!, u_l, tspan, p)
        sol2 = solve(prob2,lsoda(),dt=1e-3,saveat=1e-3)
        diff = sol2.u[2] .- sol1.u[2]
        DL = (N*sum((diff).^2)^0.5)/0.001
        u_ = sol1.u[2]
        append!(λ,log(DL))
    end
end

Lyapunov! (generic function with 1 method)

In [90]:
Lyapunov!(λ)

In [91]:
plot(λ,legend=false)
title!("Lyapunov exponens")
savefig("Lyapunov.pdf")

"/home/nadorij/Elementary_chaos_theory/Lyapunov.pdf"